# Packages

In [4]:
#from os import listdir
#from os.path import isfile, join
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [5]:
from sklearn import metrics
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

In [6]:
from sklearn.linear_model import SGDClassifier

In [7]:
import statsmodels.api as sm

C:\Users\bbnsu_000\Anaconda2\envs\project\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [8]:
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier

# Reading Data

In [10]:
mypath = './bike_data'
#onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

#onlyfiles

#onlyfiles = np.sort(onlyfiles)

#onlyfiles[2]

In [11]:
Weather = pd.read_excel('./Meteorology_NYC/Meteorology_NYC.xls', header=None)

IOError: [Errno 2] No such file or directory: './Meteorology_NYC/Meteorology_NYC.xls'

In [ ]:
Weather

In [ ]:
file_names = [str(f)+'.csv' for f in range(1, 4393)]

In [ ]:
file_names[-1]

In [ ]:
f1 = pd.read_csv(mypath+'/1.csv', header=None)

In [ ]:
f1

In [ ]:
NY_Bike_Data = pd.DataFrame(data=None, index=None, columns=["Timestamp","FromStation","ToStation","BikeMoved","Weekday","Snowy", "Rainy", "Foggy", "Sunny", "Temperature","WindSpeed"])

In [ ]:
Temp_Data = []

StartTimestamp = datetime(2014, 4, 1, 0, 0)
daysAdded = 0
for f in file_names:
    f1 = pd.read_csv(mypath+'/'+file_names[daysAdded], header=None)
    for i in range(0,len(f1)):
        for j in range(0,len(f1)):
            d = {
                "Timestamp": StartTimestamp + timedelta(hours=daysAdded),
                "FromStation": i+1,
                "ToStation": j+1,
                "BikeMoved": f1[i][j],
                "Weekday": Weather[0][daysAdded],
                "Snowy": Weather[1][daysAdded],
                "Rainy": Weather[2][daysAdded],
                "Foggy": Weather[3][daysAdded],
                "Sunny": Weather[4][daysAdded],
                "Temperature": Weather[5][daysAdded],
                "WindSpeed": Weather[6][daysAdded]
            };
            Temp_Data.append(d)
    daysAdded += 1

In [ ]:
c = StartTimestamp.time

In [ ]:
NY_Bike_Data.head()

In [ ]:
NY_Bike_Data = NY_Bike_Data.append(Temp_Data)

In [ ]:
NY_Bike_Data["Timestamp"] = pd.to_datetime(NY_Bike_Data["Timestamp"])
#df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

In [ ]:
NY_Bike_Data['Hour'] = NY_Bike_Data['Timestamp'].dt.hour

In [ ]:
NY_Bike_Data['Month'] = NY_Bike_Data['Timestamp'].dt.month

NY_Bike_Data["FromStation"] = NY_Bike_Data["FromStation"].astype("category")
NY_Bike_Data["ToStation"] = NY_Bike_Data["ToStation"].astype("category")
NY_Bike_Data["Weekday"] = NY_Bike_Data["Weekday"].astype("category")
NY_Bike_Data["Snowy"] = NY_Bike_Data["Snowy"].astype("category")
NY_Bike_Data["Rainy"] = NY_Bike_Data["Rainy"].astype("category")
NY_Bike_Data["Foggy"] = NY_Bike_Data["Foggy"].astype("category")
NY_Bike_Data["Sunny"] = NY_Bike_Data["Sunny"].astype("category")
#NY_Bike_Data["Temperature"] = NY_Bike_Data["Temperature"].astype("category")
#NY_Bike_Data["WindSpeed"] = NY_Bike_Data["WindSpeed"].astype("category")
NY_Bike_Data["Hour"] = NY_Bike_Data["Hour"].astype("category")
NY_Bike_Data["Month"] = NY_Bike_Data["Month"].astype("category")

In [ ]:
NY_Bike_Data.to_csv('./NY_Bike_Cleaned.csv')

In [ ]:
NY_Bike_Data = pd.read_csv('NY_Bike_Cleaned.csv')

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
for i in ["FromStation","ToStation","Weekday","Snowy","Rainy","Foggy","Sunny","Hour","Month"]:
    enc.fit([NY_Bike_Data[i]])
    NY_Bike_Data[i] = pd.Series(data=enc.transform([NY_Bike_Data[i]]).toarray()[0])


# Dividing data into clusters

In [ ]:
cluster1 = NY_Bike_Data[NY_Bike_Data["FromStation"] == 1]

cluster1["FromStation"] = cluster1["FromStation"].astype("category")
cluster1["ToStation"] = cluster1["ToStation"].astype("category")
cluster1["Weekday"] = cluster1["Weekday"].astype("category")
cluster1["Snowy"] = cluster1["Snowy"].astype("category")
cluster1["Rainy"] = cluster1["Rainy"].astype("category")
cluster1["Foggy"] = cluster1["Foggy"].astype("category")
cluster1["Sunny"] = cluster1["Sunny"].astype("category")
cluster1["Hour"] = cluster1["Hour"].astype("category")
cluster1["Month"] = cluster1["Month"].astype("category")

In [ ]:
cluster1.head(n=2)

In [ ]:
cluster_names = ['cluster'+str(a)+'.csv' for a in range(1, 21)]

In [ ]:
cluster_names[19]

In [ ]:
cluster_num = range(21)[1:21]

In [ ]:
cluster_num

In [ ]:
i = 0
for i in cluster_num:
    x = NY_Bike_Data[NY_Bike_Data["FromStation"] == i]
    x.to_csv(cluster_names[i-1])

In [ ]:
cluster1 = pd.read_csv("cluster1.csv")

In [ ]:
cluster1.shape

# Linear regression

In [ ]:
NY_Bike_Data[:3912*400]. head(n=5)

In [ ]:
traindata_full = NY_Bike_Data[:3912*400]
testdata_full = NY_Bike_Data[3912*400:]

In [ ]:
factors = ['FromStation', 'ToStation', 'Weekday',
       'Snowy', 'Rainy', 'Foggy', 'Sunny', 'Temperature', 'WindSpeed', 'Hour',
       'Month']
X_train = traindata_full[factors]

In [ ]:
Y_train = traindata_full['BikeMoved']

In [ ]:
X_test = testdata_full[factors]

In [ ]:
lm=linear_model.LinearRegression()
model=lm.fit(X_train,Y_train)
pred=lm.predict(X_test)

In [ ]:
x = X_train
Y = Y_train
x=sm.add_constant(x)

In [ ]:
model1 = sm.OLS(Y.astype(float), x.astype(float)).fit()
model1.summary()

In [ ]:
pred_round=pred.round()

In [ ]:
ypred= pd.Series(pred)

In [ ]:
preddf = pd.DataFrame(pred_round)

In [ ]:
preddf.describe()

In [ ]:
Y_test = testdata_full['BikeMoved']

In [ ]:
Y_test.shape

In [ ]:
metrics.accuracy_score(Y_test, pred_round)

# Linear Regression on Cluster

In [ ]:
cluster1 = pd.read_csv("cluster1.csv")

d = [{
    "Timestamp":StartTimestamp,
    "FromStation":"test",
    "ToStation":"test",
    "BikeMoved":"test",
    "Weekday":"test",
    "Snowy":"test",
    "Rainy":"test",
    "Foggy":"test",
    "Sunny":"test",
    "Temperature":"test",
    "WindSpeed":"test"
},{
    "Timestamp":StartTimestamp+ timedelta(hours=1),
    "FromStation":"test",
    "ToStation":"test",
    "BikeMoved":"test",
    "Weekday":"test",
    "Snowy":"test",
    "Rainy":"test",
    "Foggy":"test",
    "Sunny":"test",
    "Temperature":"test",
    "WindSpeed":"test"
}]
#v = pd.Series(d)
NY_Bike_Data.append(d)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
for i in ["FromStation","ToStation","Weekday","Snowy","Rainy","Foggy","Sunny","Hour","Month"]:
    enc.fit([cluster1[i]])
    cluster1[i] = pd.Series(data=enc.transform([cluster1[i]]).toarray()[0])


In [ ]:
traindata1 = cluster1[:3912*20]
testdata1 = cluster1[3912*20:]

In [ ]:
cluster1.dtypes

In [ ]:
factors = ['FromStation', 'ToStation', 'Weekday',
       'Snowy', 'Rainy', 'Foggy', 'Sunny', 'Temperature', 'WindSpeed', 'Hour',
       'Month']
X_train1 = traindata1[factors]

In [ ]:
Y_train1 = traindata1['BikeMoved']

In [ ]:
X_test1 = testdata1[factors]

In [ ]:
lm=linear_model.LinearRegression()
model1=lm.fit(X_train1,Y_train1)
pred1=lm.predict(X_test1)

In [ ]:
pred1.shape

In [ ]:
x1 = X_train1
Y1 = Y_train1
x1=sm.add_constant(x1)

In [ ]:
model11 = sm.OLS(Y1.astype(float), x1.astype(float)).fit()
model11.summary()

In [ ]:
pred1_round=pred1.round()

In [ ]:
Y_test1 = testdata1['BikeMoved']

In [ ]:
metrics.accuracy_score(Y_test1, pred1_round)

# SGD Regressor on total data

In [ ]:
#SGD Regressor
sgd_clf = SGDClassifier(loss="hinge", penalty="l2")


In [ ]:
sgd_clf.fit(X_train,Y_train)
sgd_clf.score(X_train,Y_train)

In [ ]:
sgd_predict = sgd_clf.predict(X_test)

In [ ]:
sgd_predict

In [ ]:
Y_test = testdata_full['BikeMoved']

In [ ]:
metrics.accuracy_score(Y_test, sgd_predict)

# SGD Regressor on cluster 1

In [ ]:
sgd_clf.fit(X_train1,Y_train1)
sgd_clf.score(X_train1,Y_train1)

In [ ]:
sgd_predict1 = sgd_clf.predict(X_test1)

In [ ]:
Y_test1 = testdata1['BikeMoved']

In [ ]:
metrics.accuracy_score(Y_test1, sgd_predict1)

# Neural network

In [ ]:
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train1)  
X_train1n = scaler.transform(X_train1)  
# apply same transformation to test data
X_test1n = scaler.transform(X_test1)  



In [ ]:
#from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

clf.fit(X_train1n,Y_train1)

In [ ]:
clf.score(X_train1n,Y_train1)

In [ ]:
pred1n = clf.predict(X_test1n)

In [ ]:
metrics.accuracy_score(Y_test1, pred1n)